In [7]:
!pip install spotipy





In [8]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

CLIENT_ID = os.getenv("2119e4bb9b494274b4d0bb6d16525b06")
CLIENT_SECRET = os.getenv("3a6788ae729249ceab5a07a5979aa406T")
REDIRECT_URI = os.getenv("SPOTIFY_REDIRECT_URI", "http://localhost:8501/callback")


In [10]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

def make_app_client():
    return Spotify(auth_manager=SpotifyClientCredentials())


In [11]:
# ==== HIT PREDICTOR: guaranteed-run baseline (offline by default) ====
# Skills: EDA, Feature Eng, Logistic Regression, Random Forest, XGBoost, Streamlit dashboard
# Optional: set USE_SPOTIFY=True later to try real API again.

# 0) Deps (safe to rerun)
%pip install -q pandas numpy scikit-learn xgboost streamlit matplotlib seaborn

import os, json, math, time, random, textwrap
import numpy as np, pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import joblib

# ---------------- CONFIG ----------------
USE_SPOTIFY = False  # <<< leave False for zero-errors run. Flip to True later if you want to try API again.
OUTPUT_DIR = Path("data"); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
RAW_CSV = OUTPUT_DIR / "raw_tracks.csv"
SEED = 42
random.seed(SEED); np.random.seed(SEED)

# ---------------- OPTIONAL SPOTIFY (kept off by default) ----------------
def try_spotify_build(max_rows=200, market="US"):
    """
    Very small 'best-effort' Spotify builder that won't crash the notebook:
    - Only runs if USE_SPOTIFY=True AND credentials are present
    - If anything fails, returns None so the offline path still runs
    """
    try:
        import requests, soundfile as sf, librosa
        from spotipy import Spotify
        from spotipy.oauth2 import SpotifyClientCredentials
    except Exception as e:
        print(f"ℹ️ Spotify/preview deps not available: {e}")
        return None

    CLIENT_ID = os.environ.get("2119e4bb9b494274b4d0bb6d16525b06")
    CLIENT_SECRET = os.environ.get("3a6788ae729249ceab5a07a5979aa406")
    if not CLIENT_ID or not CLIENT_SECRET:
        print("ℹ️ No SPOTIFY_CLIENT_ID/SECRET env vars. Skipping API and using offline dataset.")
        return None

    try:
        auth = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
        sp = Spotify(auth_manager=auth)
        _ = auth.get_access_token(as_dict=False)  # sanity check
        print("✅ Spotify token OK.")
    except Exception as e:
        print(f"ℹ️ Token check failed ({e}). Using offline dataset.")
        return None

    # helper: robust search for previewable tracks
    def search_preview(q, lim=50):
        out = []
        for off in range(0, lim, 20):
            try:
                res = sp.search(q=q, type="track", limit=20, offset=off, market=market)
                for t in res.get("tracks", {}).get("items", []):
                    if t.get("preview_url"):
                        out.append({
                            "track_id": t["id"],
                            "track_name": t["name"],
                            "artist_name": ", ".join(a["name"] for a in t.get("artists", [])),
                            "popularity": t.get("popularity") or 0,
                            "preview_url": t["preview_url"],
                        })
            except Exception:
                pass
            time.sleep(0.05)
        # dedup
        seen=set(); unique=[]
        for r in out:
            if r["track_id"] in seen: continue
            unique.append(r); seen.add(r["track_id"])
        return unique

    # tiny feature function
    def features_from_preview(url):
        try:
            import io, requests, librosa, numpy as _np, soundfile as sf
            r = requests.get(url, timeout=20); r.raise_for_status()
            y, sr = sf.read(io.BytesIO(r.content), dtype="float32")
            if y.ndim > 1: y = y.mean(axis=1)
            yt, _ = librosa.effects.trim(y, top_db=25)
            tempo, _ = librosa.beat.beat_track(y=yt, sr=sr)
            onset = librosa.onset.onset_strength(y=yt, sr=sr)
            rhythm = float(np.clip(np.std(onset)/3.0, 0, 1))
            rms = float(librosa.feature.rms(y=yt).mean())
            energy = float(np.clip(rms*3.0, 0, 1))
            zcr = float(librosa.feature.zero_crossing_rate(yt).mean())
            danceability = float(np.clip(0.6*zcr*2.0 + 0.4*rhythm, 0, 1))
            spec_cent = float(librosa.feature.spectral_centroid(y=yt, sr=sr).mean())
            valence = float(np.clip(spec_cent/5000.0, 0, 1))
            return {"danceability":danceability,"energy":energy,"valence":valence,"tempo":float(tempo)}
        except Exception:
            return None

    queries = [
        "genre:pop year:2018-2024",
        "genre:hip-hop year:2018-2024",
        "genre:dance year:2018-2024",
        "lofi beats",
        "edm instrumental",
        "hindi pop year:2015-2024",
    ]
    rows=[]
    for q in queries:
        rows += search_preview(q, lim=80)
        if len(rows) >= max_rows: break
    if not rows:
        print("ℹ️ Couldn’t find previewable tracks. Using offline dataset.")
        return None

    base = pd.DataFrame(rows).head(max_rows).copy()
    feats=[]
    for _, r in base.iterrows():
        f = features_from_preview(r["preview_url"])
        feats.append(f)
    feat_df = pd.DataFrame(feats)
    data = pd.concat([base.reset_index(drop=True), feat_df], axis=1)
    data = data.dropna(subset=["danceability","energy","valence","tempo"]).copy()
    if data.empty:
        print("ℹ️ No complete previews. Using offline dataset.")
        return None
    data["hit"] = (data["popularity"] >= 80).astype(int)
    return data[["track_id","track_name","artist_name","popularity","danceability","energy","valence","tempo","hit"]]

# ---------------- OFFLINE DATA (guaranteed) ----------------
def make_offline_dataset(n=600):
    """
    Generate a realistic, Spotify-shaped dataset (no network). 
    Correlations mirror typical audio-features; hit prob depends on energy, danceability, valence and mid-tempo.
    """
    rng = np.random.default_rng(SEED)
    # base features ~ beta distributions to keep 0..1
    dance = rng.beta(2.5, 2.0, n)
    energ = rng.beta(2.2, 1.8, n)
    valen = rng.beta(2.0, 2.2, n)
    tempo = rng.normal(120, 25, n).clip(60, 200)  # BPM

    # Nonlinear interactions & noise
    midtempo = np.exp(-((tempo-120.0)**2)/(2*15**2))  # peak around 120
    logit = -1.2 + 1.8*dance + 1.4*energ + 0.8*valen + 1.1*midtempo + 0.7*dance*energ - 0.5*(tempo>170)
    prob = 1/(1+np.exp(-logit))
    hit = (rng.uniform(0,1,n) < prob).astype(int)

    # Popularity roughly aligned with prob + noise (0..100)
    pop = np.clip((prob*100 + rng.normal(0,12,n)), 0, 100).round().astype(int)

    df = pd.DataFrame({
        "track_id": [f"synth_{i:05d}" for i in range(n)],
        "track_name": [f"Synth Track {i}" for i in range(n)],
        "artist_name": [f"Artist {i%20}" for i in range(n)],
        "popularity": pop,
        "danceability": dance,
        "energy": energ,
        "valence": valen,
        "tempo": tempo,
        "hit": hit,
    })
    return df

# ---------------- BUILD DATASET ----------------
if USE_SPOTIFY:
    df = try_spotify_build()
    if df is None:
        df = make_offline_dataset()
else:
    df = make_offline_dataset()

df.to_csv(RAW_CSV, index=False)
print(f"💾 Saved dataset → {RAW_CSV} ({len(df)} rows)")

# ---------------- QUICK EDA ----------------
print("\n🔎 EDA — basic stats")
print(df[["danceability","energy","valence","tempo","popularity"]].describe().round(3))
print("\nClass balance (hit=1):", round(df['hit'].mean(), 3))

# ---------------- FEATURE ENGINEERING ----------------
X = df[["danceability","energy","valence","tempo"]].copy()
# add a few simple interactions
X["dance_x_energy"] = X["danceability"] * X["energy"]
X["tempo_z"] = (X["tempo"] - X["tempo"].mean())/X["tempo"].std()
y = df["hit"].astype(int)

num_cols = X.columns.tolist()

pre = ColumnTransformer(
    transformers=[("num", StandardScaler(), num_cols)],
    remainder="drop"
)

# ---------------- TRAIN / TEST ----------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)

# 1) Logistic Regression
lr = Pipeline([
    ("pre", pre),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", random_state=SEED))
])
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_proba = lr.predict_proba(X_test)[:,1]
print("\n✅ Logistic Regression")
print("acc:", round(accuracy_score(y_test, lr_pred), 3),
      "f1:", round(f1_score(y_test, lr_pred), 3),
      "roc_auc:", round(roc_auc_score(y_test, lr_proba), 3))

# 2) Random Forest
rf = Pipeline([
    ("pre", pre),
    ("clf", RandomForestClassifier(
        n_estimators=300, max_depth=None, min_samples_leaf=2,
        class_weight="balanced_subsample", random_state=SEED, n_jobs=-1))
])
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_proba = rf.predict_proba(X_test)[:,1]
print("\n✅ Random Forest")
print("acc:", round(accuracy_score(y_test, rf_pred), 3),
      "f1:", round(f1_score(y_test, rf_pred), 3),
      "roc_auc:", round(roc_auc_score(y_test, rf_proba), 3))

# 3) XGBoost
xgb_clf = xgb.XGBClassifier(
    n_estimators=500, learning_rate=0.05, max_depth=5,
    subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
    objective="binary:logistic", eval_metric="auc", random_state=SEED, n_jobs=-1
)
xgb_pipe = Pipeline([("pre", pre), ("clf", xgb_clf)])
xgb_pipe.fit(X_train, y_train)
xgb_pred = xgb_pipe.predict(X_test)
xgb_proba = xgb_pipe.predict_proba(X_test)[:,1]
print("\n✅ XGBoost")
print("acc:", round(accuracy_score(y_test, xgb_pred), 3),
      "f1:", round(f1_score(y_test, xgb_pred), 3),
      "roc_auc:", round(roc_auc_score(y_test, xgb_proba), 3))

# ---------------- SAVE MODELS ----------------
Path("models").mkdir(exist_ok=True)
joblib.dump(lr, "models/model_logreg.joblib")
joblib.dump(rf, "models/model_rf.joblib")
joblib.dump(xgb_pipe, "models/model_xgb.joblib")
print("\n💾 Saved models → models/model_*.joblib")

# ---------------- WRITE STREAMLIT APP ----------------
app_code = f"""
import joblib, pandas as pd, numpy as np
import streamlit as st

st.set_page_config(page_title="Spotify Hit Predictor (Demo)", page_icon="🎵", layout="centered")

st.title("🎵 Spotify Hit Predictor — Demo")
st.markdown("This demo uses features similar to Spotify audio features (danceability, energy, valence, tempo).")

df = pd.read_csv("{RAW_CSV.as_posix()}")
st.caption(f"Dataset rows: {{len(df)}}")

with st.expander("Peek at data"):
    st.dataframe(df.head(20))

# Load models
lr = joblib.load("models/model_logreg.joblib")
rf = joblib.load("models/model_rf.joblib")
xgb = joblib.load("models/model_xgb.joblib")

st.subheader("Try your own track features")
danceability = st.slider("Danceability", 0.0, 1.0, 0.6, 0.01)
energy       = st.slider("Energy",       0.0, 1.0, 0.6, 0.01)
valence      = st.slider("Valence",      0.0, 1.0, 0.5, 0.01)
tempo        = st.slider("Tempo (BPM)",  60.0, 200.0, 120.0, 1.0)

# feature eng
row = pd.DataFrame([{{"danceability":danceability, "energy":energy, "valence":valence, "tempo":tempo}}])
row["dance_x_energy"] = row["danceability"]*row["energy"]
row["tempo_z"] = (row["tempo"] - df["tempo"].mean())/df["tempo"].std()

proba_lr  = float(lr.predict_proba(row)[0,1])
proba_rf  = float(rf.predict_proba(row)[0,1])
proba_xgb = float(xgb.predict_proba(row)[0,1])

st.write("### Predicted Hit Probability")
st.write(f"LogReg: **{{proba_lr:.2f}}**  |  RF: **{{proba_rf:.2f}}**  |  XGB: **{{proba_xgb:.2f}}**")

st.markdown("---")
st.subheader("Compare your taste vs. 'Top hits' in this dataset")
hits = df[df["hit"]==1]
yours = row.assign(popularity=0, hit=0)
col1, col2 = st.columns(2)
with col1:
    st.metric("Avg Danceability (hits)", f"{{hits['danceability'].mean():.2f}}")
    st.metric("Avg Energy (hits)", f"{{hits['energy'].mean():.2f}}")
with col2:
    st.metric("Your Danceability", f"{{danceability:.2f}}")
    st.metric("Your Energy", f"{{energy:.2f}}")

st.caption("Tip: To switch to real Spotify collection later, open the notebook and set USE_SPOTIFY=True.")
"""
Path("app.py").write_text(app_code, encoding="utf-8")
print("Streamlit app written → app.py")
print("\nNext steps:\n  1) Run:  streamlit run app.py\n  2) Move the sliders to see LR/RF/XGB probabilities.\n")




Note: you may need to restart the kernel to use updated packages.
💾 Saved dataset → data/raw_tracks.csv (600 rows)

🔎 EDA — basic stats
       danceability   energy  valence    tempo  popularity
count       600.000  600.000  600.000  600.000     600.000
mean          0.556    0.553    0.479  117.991      81.738
std           0.218    0.219    0.210   24.297      15.082
min           0.019    0.020    0.032   60.000      16.000
25%           0.395    0.393    0.319  101.992      73.000
50%           0.562    0.573    0.470  117.149      84.000
75%           0.728    0.730    0.627  134.083      94.000
max           0.979    0.991    0.987  197.918     100.000

Class balance (hit=1): 0.817

✅ Logistic Regression
acc: 0.692 f1: 0.778 roc_auc: 0.791

✅ Random Forest
acc: 0.767 f1: 0.865 roc_auc: 0.67

✅ XGBoost
acc: 0.775 f1: 0.871 roc_auc: 0.591

💾 Saved models → models/model_*.joblib
Streamlit app written → app.py

Next steps:
  1) Run:  streamlit run app.py
  2) Move the sliders to see 

In [ ]:
! streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.11:8501

